1. En este paso, el banco nos envía la carta


In [ ]:
import win32com.client
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from datetime import datetime
from pathlib import Path
import shutil

# Fecha actual
today = datetime.now()
year = today.strftime("%Y")
month = today.strftime("%m")
day = today.strftime("%d%m%y")

# Crear carpeta de salida
base_dir = Path(r"Z:\17. Reporting Automation\Cartas NDFs\Cartas sin firmas")
output_dir = base_dir / year / month / day
output_dir.mkdir(parents=True, exist_ok=True)

# Conexión con Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.Folders("Mercado de Capitales Colombia").Folders("Cartas NDF")
messages = inbox.Items
messages.Sort("[ReceivedTime]", True)

# Rango de tiempo: desde el inicio del día hasta ahora
#start = datetime(today.year, today.month, today.day)

# Acá si necesito procesar todos los correos del mes
start = datetime(today.year, today.month, 1)

# La fecha si es siempre el día de hoy
end = today

messages_today = messages.Restrict(
    "[ReceivedTime] >= '" + start.strftime("%m/%d/%Y %I:%M %p") +
    "' AND [ReceivedTime] < '" + end.strftime("%m/%d/%Y %I:%M %p") + "'"
)

# Procesar correos con adjuntos
if messages_today.Count == 0:
    print("No hay correos recibidos hoy con adjuntos.")
else:
    for message in messages_today:
        received = message.ReceivedTime.replace(tzinfo=None)
        if start <= received <= end:

            attachments = message.Attachments
            for attachment in attachments:
                filename = attachment.FileName.lower()
                if filename.endswith(".pdf") or filename.endswith(".xlsx"):
                    safe_filename = re.sub(r'[^0-9a-zA-ZáéíóúÁÉÍÓÚñÑ\.\-_ ]+', '', attachment.FileName)
                    attachment.SaveAsFile(output_dir / safe_filename)
                    print(f"Adjunto guardado: {safe_filename}")

# Procesar correos con enlaces de descarga
for message in messages_today:
    if message.Subject.startswith("JPM Confirmation ID"):
        received = message.ReceivedTime.replace(tzinfo=None)
        if start <= received <= end:

            body = message.Body

            # Extraer URL y código de verificación
            url_match = re.search(r"https://[^\s]+", body)
            code_match = re.search(r"Verification Code:\s*([A-Z0-9]+)", body)

            if url_match and code_match:
                url = url_match.group(0)
                code = code_match.group(1)

                # Abrir navegador y automatizar ingreso del código
                driver = webdriver.Chrome()
                driver.get(url)

                time.sleep(3)

                try:
                    input_box = driver.find_element(By.ID, "txtAccessCode")
                    input_box.send_keys(code)
                    input_box.send_keys(Keys.RETURN)
                    print("Código ingresado correctamente.")
                    time.sleep(3)

                    download_link = driver.find_element(By.CLASS_NAME, "Q416_downloadspecific")
                    download_link.click()
                    print("Descarga iniciada correctamente.")
                except Exception as e:
                    print("Error durante la automatización del navegador:", e)

                time.sleep(2)
                driver.quit()

                # Mover archivos PDF descargados hoy
                downloads_dir = Path(r"C:\Users\ntorreslo\Downloads")
                for file in downloads_dir.glob("Confirmation-AE*.pdf"):
                    if file.is_file():
                        modified_time = datetime.fromtimestamp(file.stat().st_mtime)
                        if modified_time.date() == today.date():
                            destination = output_dir / file.name
                            shutil.move(str(file), str(destination))
                            print(f"Archivo movido: {file.name}")


Adjunto guardado: 8301225661_20250716_NDFV_FW261505.pdf
Adjunto guardado: 8301225661_20250724_NDFC_FW261945.pdf
Adjunto guardado: 1307699.pdf
Adjunto guardado: 1305459.pdf
Adjunto guardado: 8301225661_20250716_NDFV_FW261505.pdf
Adjunto guardado: 8301225661_20250724_NDFC_FW261945.pdf
Adjunto guardado: 1304173.pdf
Adjunto guardado: 1304176.pdf
Adjunto guardado: 8301225661_20250724_NDFC_FW261945.pdf
Adjunto guardado: 1305459.pdf
Adjunto guardado: 1305459.pdf
Adjunto guardado: 8301225661_20250716_NDFV_FW261505.pdf
Adjunto guardado: 8301225661_20250716_NDFV_FW261505.pdf
Adjunto guardado: 1304173.pdf
Adjunto guardado: 1304176.pdf
Adjunto guardado: 8301225661_20250716_NDFV_FW261505.pdf
Adjunto guardado: Bancol 12345.pdf
Adjunto guardado: Bancol 123456.pdf
Adjunto guardado: 1296252.pdf
Adjunto guardado: 1296252.pdf
Código ingresado correctamente.
Descarga iniciada correctamente.
Archivo movido: Confirmation-AE2025073058688986.pdf
Código ingresado correctamente.
Descarga iniciada correctamente.

In [1]:
import win32com.client

# Conexión con Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Acceder a la carpeta específica
inbox = outlook.Folders("Mercado de Capitales Colombia").Folders("Cartas NDF")

# Obtener todos los correos
messages = inbox.Items
messages.Sort("[ReceivedTime]", True)

# Filtrar correos cuyo asunto comienza con "JPM Confirmation ID"
for message in messages:
    if message.Subject.startswith("JPM Confirmation ID"):
        print(f"Asunto: {message.Subject}")
        print("Contenido del correo:")
        print(message.Body)
        print("-" * 80)


Asunto: JPM Confirmation ID AE2025071091434370—COLOMBIA TELECOMUNICACIONES SA ESP BIC
Contenido del correo:
AVISO/WARNING: Este correo electrónico se originó desde fuera de la organización. No haga clic en enlaces ni abra archivos adjuntos a menos que reconozca al remitente y sepa que el contenido es seguro / This email has been originated from outside of the organization. Do not click links or open attachments unless you recognize the sender and know the content is safe. 

 	
New Confirmation Pending Review 

A new confirmation document is ready. Please confirm core economics are correct. 

To review, click the link—then enter the access code. 

J.P. Morgan Confirmation <https://jpmorgan.eu9.smartcommunications.cloud/produce/wizard/e023906b-5f2e-47be-9c43-a3a3da21cec5/?workflowId=c4f19701-f94f-897a-bc89-3b39a6cd358a&tu=1> 

Verification Code: W54G91JSZ3 

Questions or issues? Contact: london.fx.confirmations@jpmorgan.com <mailto:london.fx.confirmations@jpmorgan.com> 

If you are not t

2. En el punto anterior ya se guarda el pdf en la carpeta

3. En el Excel de seguimiento de las cartas, hacer las operaciones necesarias 

In [ ]:
import os
import re
import pandas as pd
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path 

# Configurar la ruta de Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\ntorreslo\AppData\Local\Programs\Tesseract-OCR\tesseract'

def extract_data_from_pdf(file_path):
    # Open the PDF file
    #pdf_document = fitz.open(file_path)
    #text = ""
    # Extract text from each page
    #for page_num in range(len(pdf_document)):
    #    page = pdf_document.load_page(page_num)
    #    text += page.get_text()
    images = convert_from_path(file_path
                               , dpi = 500
                               , poppler_path=r'C:\Poppler\poppler-24.08.0\Library\bin')  
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image, lang='spa')  # Cambia 'spa' por el idioma deseado

    # Define regex patterns for both Spanish and English documents
    """
    patterns = {
        "fecha_celebracion": r"(?i)(Fecha de Celebración|Trade Date):\s*([\d]{1,2}\s\w+\s\d{4}|\d{1,2}\s\w+\s\d{4})",
        "fecha_vencimiento": r"(?i)(Fecha de Vencimiento|Valuation Date):\s*([\d]{1,2}\s\w+\s\d{4}|\d{1,2}\s\w+\s\d{4})",
        "fecha_liquidacion": r"(?i)(Fecha de Liquidación|Settlement Date):\s*([\d]{1,2}\s\w+\s\d{4}|\d{1,2}\s\w+\s\d{4})",
        "tasa_forward": r"(?i)(Tasa Forward|Tasa|Forward Rate):\s*([\d,\.]+)",
        "valor_nominal": r"(?i)(Valor Nominal|Notional Amount|Valor Negociado):\s*USD\s*([\d,\.]+)",
        "contraparte": r"(?i)(Vendedor|Seller|Parte A):\s*([\w\s]+)",
        "tipo_operacion": r"(?i)(Tipo de Operación|Type of Operation):\s*([\w\s]+)"
    }
    """
    patterns = {
        "fecha_celebracion": r"(?i)(Fecha de Celebraci[oó]n|Trade Date):?\s*(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{4}|\d{1,2}\s\w+\s\d{4}|\w+\s\d{1,2}\sdel\s\d{4})",
        "fecha_vencimiento": r"(?i)(Fecha de Vencimiento|Fixing Date|Valuation Date|F\.? VEN):?\s*(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{4}|\d{1,2}\s\w+\s\d{4}|\w+\s\d{1,2}\sdel\s\d{4})",
        "fecha_liquidacion": r"(?i)(Fecha de Liquidaci[oó]n|Fecha de Cumplimiento|Settlement Date|F\.? CUM|Fecha Compensaci[oó]n):?\s*(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{4}|\d{1,2}\s\w+\s\d{4}|\w+\s\d{1,2}\sdel\s\d{4})",
        "tasa_forward": r"(?i)(Tasa Forward|Forward Rate|Tasa):?\s*\$?\s*([\d\.,]+)",
        "valor_nominal": r"(?i)(Valor Nominal|Notional Amount|Valor Negociado| Monto en USD):\s*USD\s*([\d,\.]+)",
        #"contraparte": r"(?i)(Parte A|Vendedor|Seller|Reference Currency Seller):?\s*([A-ZÁÉÍÓÚÑa-z\s\.\-]+)",
        "contraparte": r"(?i)(SANTANDER|BANCOLOMBIA|JPMorgan|OCCIDENTE|DAVIVIENDA|BANCO DE BOGOT[AÁ]|CITIBANK|SCOTIABANK)",
        "tipo_operacion": r"(?i)(Tipo de Operación|Type of Operation):\s*([\w\s]+)"
    }
      

    data = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            data[key] = match.group(2).strip() if match.lastindex >= 2 else match.group(1).strip()
        else:
            data[key] = None

    # Clean the forward rate to retain only digits, commas, and periods
    if data["tasa_forward"]:
        data["tasa_forward"] = re.sub(r"[^\d,\.]", "", data["tasa_forward"])

    return data

def process_pdfs_in_directory(directory):
    records = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            data = extract_data_from_pdf(file_path)
            data["nombre_archivo"] = filename
            records.append(data)
    return records

# Directory containing the PDFs
directory = r"Z:/17. Reporting Automation/Cartas NDFs/Cartas sin firmas"

# Process the PDFs and get the data
records = process_pdfs_in_directory(directory)

# Create a DataFrame and save to Excel
df = pd.DataFrame(records)
df.to_excel(r"Z:/17. Reporting Automation/Cartas NDFs/forward_tracking.xlsx", index=False)

print("The data has been successfully extracted and saved to forward_tracking.xlsx.")


In [ ]:
import os
import re
import pandas as pd
import pytesseract
from pdf2image import convert_from_path 

# Configurar la ruta de Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\ntorreslo\AppData\Local\Programs\Tesseract-OCR\tesseract'

def extract_data_from_pdf(file_path):
    # Open the PDF file
    #pdf_document = fitz.open(file_path)
    #text = ""
    # Extract text from each page
    #for page_num in range(len(pdf_document)):
    #    page = pdf_document.load_page(page_num)
    #    text += page.get_text()
    images = convert_from_path(file_path
                               , dpi = 500
                               , poppler_path=r'C:\Poppler\poppler-24.08.0\Library\bin')  
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image, lang='spa')  # Cambia 'spa' por el idioma deseado

    # Define regex patterns for both Spanish and English documents
    patterns = {
        "fecha_celebracion": [
    r"(?:Fecha(?: de Celebraci[oó]n| de Vencimiento| de celebración)?|Trade Date).*?[:\-]?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{1,2}\s+de\s+\w{3,9}\.?(\s+de)?\s+\d{4})",
    r"\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b",
    r"\b\d{1,2}\s+de\s+\w{3,9}\.?(\s+de)?\s+\d{4}\b",
    r"(?i)(Fecha de Celebraci[oó]n|Trade Date):?\s*(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{4}|\d{1,2}\s\w+\s\d{4}|\w+\s\d{1,2}\sdel\s\d{4})"
    ],
        "tasa_forward": [
                   r"Forward Rate:\s*([\d.,]+)"
                 , r"Tasa Forward\s*[-:]?\s*([\d.,]+)"
                 , r"Tasa Forward\s*[-:]?\s*([\d.,]+)"
                 , r"(?i)(Tasa Forward|Forward Rate|Tasa):?\s*\$?\s*([\d\.,]+)"
                 , r'(?:Tasa Forward|Forward Rate|Tasa FW).*?[:\s].*?[\$\s]*([\d,]+\.?\d*)'
                 , r'(\d{4}[\.,]\d{2})\s*COP/USD'
                 , r'[\|\s](\d{4}[\.,]\d{2})[\|\s]'
                 , r'(\b\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})\b)'
                 , r"(?:por encima de)?\s*\$?\s*([\d]{1,3}(?:[.,]\d{3})*(?:[.,]\d{2}))"

                 ],
        "valor_nominal": [
    r"(?:Valor(?: Nocional| Nominal)?|Notional Amount|Monto en USD|Valor Negociado).*?:?\s*USD\s*([\d.,]+)",
    r"(?i)(Valor Nominal|Notional Amount|Valor Negociado):\s*USD\s*([\d,\.]+)",
    r"USD\s*([\d.,]+)",
    r"(?:nocional|nominal|monto|valor)\s*[:\-]?\s*\$?\s?\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?"
    ],
        "contraparte": [
                    r"JPMorgan, N\.A\."
                  #, r"SCOTIABANK\.A\."
                  , r'(SCOTIABANK)'
                  , r'(DAVIVIENDA)'
                  , r'(ITAÚ)'
                  , r'(BANCOLOMBIA)'
                  , r'(JPMorgan)'
                  , r'(BANCO DE OCCIDENTE)'
                  , r'(BANCO SANTANDER)'
                  #, r'(BANCO SANTANDER[^,]*)'
                  , r'(CITIBANK COLOMBIA)'
                  ]
        
    }
      

    data = {}
    for key, pattern_list in patterns.items():
        match = None
        for pattern in pattern_list:
            match = re.search(pattern, text)
            if match:
                if match.lastindex:  # Verifica si hay grupos capturados
                    # Usa el último grupo capturado (el más específico)
                    data[key] = match.group(match.lastindex).strip()
                else:
                    data[key] = match.group(0).strip()  # Usa todo el match si no hay grupos
                break
            if not match:
                data[key] = None


    # Clean the forward rate to retain only digits, commas, and periods
    if data["tasa_forward"]:
        data["tasa_forward"] = re.sub(r"[^\d,\.]", "", data["tasa_forward"])

    return data

def process_pdfs_in_directory(directory, output_directory):
    records = []
    current_id = 1001
    today = pd.Timestamp.today().strftime('%d-%m-%y')
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            data = extract_data_from_pdf(file_path)
            data["nombre_archivo"] = filename
            data["id"] = current_id
            data["fecha_proceso"] = today

            contraparte = data.get("contraparte", "")
            fecha_proceso = data.get("fecha_proceso", today)
            nuevo_nombre = f"{contraparte} {fecha_proceso} {current_id}.pdf"
            data['nuevo_nombre_archivo'] = nuevo_nombre

            new_file_path = os.path.join(output_directory, nuevo_nombre)
            # Save the PDF with the new name   
            os.rename(file_path, new_file_path)


            records.append(data)
            current_id +=1
    return records

# Directory containing the PDFs
directory = r"Z:/17. Reporting Automation/Cartas NDFs/Cartas sin firmas"
output_directory = r"Z:/17. Reporting Automation/Cartas NDFs/Cartas para firmar"

# Process the PDFs and get the data
records = process_pdfs_in_directory(directory, output_directory)

# Create a DataFrame and save to Excel
df = pd.DataFrame(records)
df.to_excel(r"Z:/17. Reporting Automation/Cartas NDFs/forward_tracking_4.xlsx", index=False)

print("The data has been successfully extracted and saved to forward_tracking.xlsx.")

In [ ]:
import os
import re
import pandas as pd
import pytesseract
from pdf2image import convert_from_path

# Configurar la ruta de Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\ntorreslo\AppData\Local\Programs\Tesseract-OCR\tesseract'

# Patrones de extracción
tasa_patterns = [
    r"Forward Rate:\s*([\d.,]+)",
    r"Tasa Forward\s*[-:]?\s*([\d.,]+)",
    r"(?i)(Tasa Forward|Forward Rate|Tasa):?\s*\$?\s*([\d\.,]+)",
    r'(?:Tasa Forward|Forward Rate|Tasa FW).*?[:\s].*?[\$\s]*([\d,]+\.?\d*)',
    r'(\d{4}[\.,]\d{2})\s*COP/USD',
    r'[\|\s](\d{4}[\.,]\d{2})[\|\s]',
    r'(\b\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})\b)'
]

valor_patterns = [
    r"(?:Valor(?: Nocional| Nominal)?|Notional Amount|Monto en USD|Valor Negociado).*?:?\s*USD\s*([\d.,]+)",
    r"USD\s*([\d.,]+)",
    r"(?:nocional|nominal|monto|valor)\s*[:\-]?\s*\$?\s?\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?"
]

fecha_patterns = [
    r"(?:Fecha(?: de Celebraci[oó]n| de Vencimiento| de celebración)?|Trade Date).*?[:\-]?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{1,2}\s+de\s+\w{3,9}\.?(\s+de)?\s+\d{4})",
    r"\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b",
    r"\b\d{1,2}\s+de\s+\w{3,9}\.?(\s+de)?\s+\d{4}\b"
]

banco_patterns = [
    r"JPMorgan, N\.A\.",
    r"SCOTIABANK\.A\.",
    r"(SCOTIABANK COLPATRIA S\.A\.)",
    r"(DAVIVIENDA)",
    r"(ITAÚ COLOMBIA S\.A\.)",
    r"(BANCOLOMBIA S\.A\.)",
    r"(JPMorgan)",
    r"(BANCO DE OCCIDENTE)",
    r"(BANCO SANTANDER[^,]*)",
    r"(CITIBANK COLOMBIA)"
]

# Función para extraer un campo con múltiples patrones
def extract_field(patterns, text, label):
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(1).strip()
    return f"{label} no encontrado"

# Función para procesar un archivo PDF y extraer los campos
def extract_data_from_pdf(file_path):
    images = convert_from_path(file_path, dpi=500, poppler_path=r'C:\Poppler\poppler-24.08.0\Library\bin')
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image, lang='spa')

    tasa = extract_field(tasa_patterns, text, "Tasa Forward")
    valor = extract_field(valor_patterns, text, "Valor Nominal")
    fecha = extract_field(fecha_patterns, text, "Fecha de Celebración")
    banco = extract_field(banco_patterns, text, "Banco")

    return {
        "Tasa Forward": tasa,
        "Valor Nominal": valor,
        "Fecha de Celebración": fecha,
        "Banco": banco
    }

# Ruta del directorio con los archivos PDF
pdf_directory = r"Z:/17. Reporting Automation/Cartas NDFs/Cartas sin firmas"
resultados = []

# Procesar todos los archivos PDF en el directorio
for filename in os.listdir(pdf_directory):
    if filename.lower().endswith(".pdf"):
        file_path = os.path.join(pdf_directory, filename)
        data = extract_data_from_pdf(file_path)
        data["Archivo"] = filename
        resultados.append(data)

# Crear un DataFrame y exportar a Excel
df = pd.DataFrame(resultados)
df.to_excel(r"Z:/17. Reporting Automation/Cartas NDFs/forward_tracking_3.xlsx", index=False)

print("Extracción completada. Resultados guardados en 'resultados_extraccion.xlsx'.")



In [ ]:
import json
import pandas as pd
import ollama

# Ruta del archivo de texto
#file_path = r"Z:\17. Reporting Automation\Cartas NDFs\texto_extraido_bancolombia.txt"
file_path = r"Z:\17. Reporting Automation\Cartas NDFs\texto_extraido_citi.txt"

# Leer el contenido del archivo
with open(file_path, 'r', encoding='utf-8') as f:
    contrato_texto = f.read()

# Construir el prompt

prompt = f"""
Eres un experto en análisis de contratos financieros. Extrae ÚNICAMENTE los siguientes campos del texto del contrato de forward:

CAMPOS REQUERIDOS:
- banco: Nombre completo de la entidad bancaria (buscar en encabezados, firmas, o menciones como "El Banco")
- tasa_fwd: Tasa forward exacta (formato: usar coma como separador decimal, sin espacios)
- valor_nominal_usd: Monto en USD sin formato (solo números y coma decimal, eliminar puntos de miles)
- fecha_inicio: Fecha de negociación/formalización de la operación (formato: dd/mm/aaaa)

INSTRUCCIONES DE FORMATO:
- Responde SOLO con un JSON válido
- No incluyas explicaciones, comentarios, ni texto adicional
- No uses bloques de código ni etiquetas
- Usa comillas dobles para las claves y valores
- Si no encuentras un campo, usa null

EJEMPLO DE SALIDA ESPERADA:
{{
  "banco": "BANCOLOMBIA S.A.",
  "tasa_fwd": "4236,20",
  "valor_nominal_usd": "3018127,00",
  "fecha_inicio": "25/09/2024"
}}

GUÍAS DE BÚSQUEDA:
- banco: Buscar en encabezados, menciones de "El Banco", firmas, o entidades que venden/compran
- tasa_fwd: Buscar "Tasa Forward", "Forward Rate", o similar
- valor_nominal_usd: Buscar "Monto en USD", "Valor Nominal", "Amount" seguido de cifras
- fecha_inicio: Buscar fecha de negociación, formalización, o "el [fecha] del [año]"

TEXTO DEL CONTRATO:
\"\"\"{contrato_texto}\"\"\"
"""

# Llamar al modelo Ollama local
#respuesta = ollama.chat(model="qwen3:0.6b", messages=[{"role": "user", "content": prompt}])
respuesta = ollama.chat(model="llama3.2:3b", messages=[{"role": "user", "content": prompt}])

print(respuesta['message']['content'])
# Convertir la respuesta a diccionario
#datos_extraidos = json.loads(respuesta['message']['content'])

# Guardar en Excel
#df = pd.DataFrame([datos_extraidos])
#df.to_excel("resultado_forward.xlsx", index=False)

#print("Resultado guardado en 'resultado_forward.xlsx'")


In [ ]:
import ollama
import json
import re
from pathlib import Path
import fitz  # PyMuPDF
import pdfplumber
from typing import Dict, Optional

class ContractExtractor:
    def __init__(self, model="llama3.2:3b"):
        self.model = model
    
    def extract_text_from_pdf(self, pdf_path: str, method="pdfplumber") -> str:
        """
        Extrae texto de PDF usando diferentes métodos
        """
        try:
            if method == "pdfplumber":
                return self._extract_with_pdfplumber(pdf_path)
            elif method == "pymupdf":
                return self._extract_with_pymupdf(pdf_path)
            else:
                raise ValueError("Método no soportado")
        except Exception as e:
            print(f"Error extrayendo texto: {e}")
            return None
    
    def _extract_with_pdfplumber(self, pdf_path: str) -> str:
        """Extrae texto usando pdfplumber (mejor para tablas)"""
        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text
    
    def _extract_with_pymupdf(self, pdf_path: str) -> str:
        """Extrae texto usando PyMuPDF (más rápido)"""
        text = ""
        doc = fitz.open(pdf_path)
        for page_num in range(doc.page_count):
            page = doc[page_num]
            text += page.get_text() + "\n"
        doc.close()
        return text
    
    def extract_contract_data(self, pdf_path: str) -> Dict:
        """
        Extrae datos del contrato desde PDF
        """
        # Verificar si el archivo existe
        if not Path(pdf_path).exists():
            return {"error": f"Archivo no encontrado: {pdf_path}"}
        
        # Extraer texto del PDF
        print("Extrayendo texto del PDF...")
        texto_contrato = self.extract_text_from_pdf(pdf_path)
        
        if not texto_contrato:
            return {"error": "No se pudo extraer texto del PDF"}
        
        # Limpiar texto
        texto_limpio = self._clean_text(texto_contrato)
        
        # Crear prompt optimizado
        prompt = f"""
Eres un asistente especializado en análisis de contratos financieros.

Extrae ÚNICAMENTE estos campos del texto del contrato y responde con JSON limpio:

- banco: Nombre completo del banco (buscar en encabezados, firmas, "El Banco")
- tasa_fwd: Tasa Forward (formato con coma decimal: ej. 4236,20)
- valor_nominal_usd: Monto en USD (solo números, sin puntos de miles)
- fecha_inicio: Fecha de operación/negociación (formato dd/mm/aaaa)

IMPORTANTE: 
- Para tasa_fwd: Si ves 4.236,20 debe ser 4236,20
- Para valor_nominal_usd: Si ves 2,000,000 debe ser 2000000
- Responde SOLO con el JSON, sin explicaciones adicionales.

Texto del contrato:
{texto_limpio[:3000]}
"""
        
        try:
            print("Procesando con Ollama...")
            respuesta = ollama.chat(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                options={
                    "temperature": 0.1,
                    "top_p": 0.9,
                    "repeat_penalty": 1.1,
                    "num_predict": 300
                }
            )
            
            # Limpiar y parsear respuesta
            content = respuesta['message']['content'].strip()
            return self._parse_json_response(content)
            
        except Exception as e:
            return {"error": f"Error procesando con Ollama: {str(e)}"}
    
    def _clean_text(self, text: str) -> str:
        """Limpia el texto extraído del PDF"""
        # Eliminar saltos de línea excesivos
        text = re.sub(r'\n+', '\n', text)
        # Eliminar espacios excesivos
        text = re.sub(r' +', ' ', text)
        # Eliminar caracteres especiales problemáticos
        text = re.sub(r'[^\w\s\.,:/\-()$]', '', text)
        return text.strip()
    
    def _parse_json_response(self, content: str) -> Dict:
        """Parsea la respuesta JSON del modelo"""
        try:
            # Buscar JSON en la respuesta
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                resultado = json.loads(json_str)
                
                # Validar campos requeridos
                campos_req = ['banco', 'tasa_fwd', 'valor_nominal_usd', 'fecha_inicio']
                if all(campo in resultado for campo in campos_req):
                    return resultado
                else:
                    return {"error": "Campos faltantes in JSON", "data": resultado}
            else:
                return {"error": "No se encontró JSON válido", "raw_response": content}
        except json.JSONDecodeError as e:
            return {"error": f"Error parseando JSON: {str(e)}", "raw_response": content}
    
    def process_multiple_pdfs(self, pdf_directory: str) -> Dict:
        """
        Procesa múltiples PDFs en un directorio
        """
        results = {}
        pdf_dir = Path(pdf_directory)
        
        if not pdf_dir.exists():
            return {"error": f"Directorio no encontrado: {pdf_directory}"}
        
        pdf_files = list(pdf_dir.glob("*.pdf"))
        
        for pdf_file in pdf_files:
            print(f"Procesando: {pdf_file.name}")
            results[pdf_file.name] = self.extract_contract_data(str(pdf_file))
        
        return results

# Función principal para uso directo
def extraer_de_pdf(pdf_path: str, model="llama3.2:3b") -> Dict:
    """
    Función simple para extraer datos de un PDF
    """
    extractor = ContractExtractor(model=model)
    return extractor.extract_contract_data(pdf_path)

# Ejemplo de uso
if __name__ == "__main__":
    # Tu archivo específico
    #pdf_path = r"Z:\11_Gersi\Contratos Firmados\CITI\Citi 26 Nov 19915 Firmado.pdf"
    pdf_path = r"Z:\17. Reporting Automation\Cartas NDFs\BANCO SANTANDER 17-06-25 1009.pdf"
    
    # Extraer datos
    resultado = extraer_de_pdf(pdf_path)
    
    # Mostrar resultado
    print(json.dumps(resultado, indent=2, ensure_ascii=False))
    
    # Ejemplo para procesar múltiples PDFs
    # extractor = ContractExtractor()
    # resultados = extractor.process_multiple_pdfs(r"Z:\11_Gersi\Contratos Firmados\CITI")
    # print(json.dumps(resultados, indent=2, ensure_ascii=False))

In [ ]:
import os
import re
import json
import shutil
import pytesseract
import pandas as pd
from datetime import datetime
from pathlib import Path
from pdf2image import convert_from_path
from PIL import Image
import ollama

# Configurar Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\ntorreslo\AppData\Local\Programs\Tesseract-OCR\tesseract'

# Fecha actual
today = datetime.now()
year = today.strftime("%Y")
month = today.strftime("%m")
day = today.strftime("%d%m%y")
fecha_proceso = today.strftime('%d-%m-%y')

# Directorios
input_dir = Path(r"Z:/17. Reporting Automation/Cartas NDFs/Cartas sin firmas") / year / month / day
output_base_dir = Path(r"Z:/17. Reporting Automation/Cartas NDFs/Cartas para firmar")
excel_output_dir = Path(r"Z:/17. Reporting Automation/Cartas NDFs")
excel_output_dir.mkdir(parents=True, exist_ok=True)

# Patrones para extraer el banco
banco_patterns = [
    r"JPMorgan, N\.A\.",
    r"(SCOTIABANK)",
    r"(DAVIVIENDA)",
    r"(ITAÚ)",
    r"(BANCOLOMBIA)",
    r"(JPMorgan)",
    r"(BANCO DE OCCIDENTE)",
    r"(BANCO SANTANDER)",
    r"(CITIBANK COLOMBIA)"
]

# Función para extraer el nombre del banco usando OCR
def extract_banco_from_pdf(pdf_path):
    try:
        images = convert_from_path(
            pdf_path,
            dpi=500,
            poppler_path=r'C:\Poppler\poppler-24.08.0\Library\bin'
        )
        text = ""
        for image in images:
            text += pytesseract.image_to_string(image, lang='spa')
        for pattern in banco_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1).strip().upper()
        return "BANCO_DESCONOCIDO"
    except Exception as e:
        print(f"Error extrayendo banco de {pdf_path}: {e}")
        return "ERROR"

# Clase para extracción con LLM
class ContractExtractor:
    def __init__(self, model="llama3.2:3b"):
        self.model = model

    def extract_contract_data(self, pdf_path: str) -> dict:
        try:
            images = convert_from_path(
                pdf_path,
                dpi=500,
                poppler_path=r'C:\Poppler\poppler-24.08.0\Library\bin'
            )
            text = ""
            for image in images:
                text += pytesseract.image_to_string(image, lang='spa+eng')

            texto_limpio = re.sub(r'\n+', '\n', text)
            texto_limpio = re.sub(r' +', ' ', texto_limpio)
            texto_limpio = re.sub(r'[^\w\s\.,:/\-()$áéíóúüñÁÉÍÓÚÜÑ]', '', texto_limpio).strip()

            prompt = f"""
Eres un asistente especializado en análisis de contratos financieros.

Extrae ÚNICAMENTE estos campos del texto del contrato y responde con JSON limpio:

- tasa_fwd: Tasa Forward SIN PUNTOS DE MILES (formato: 4236,20 NO 4.236,20)
- valor_nominal_usd: Monto en USD (solo números, sin puntos ni comas)
- fecha_inicio: Fecha de operación/negociación (formato dd/mm/aaaa)

IMPORTANTE: 
- Para tasa_fwd: Si ves 4.236,20 debe ser 4236,20
- Para valor_nominal_usd: Si ves 2,000,000 debe ser 2000000
- Responde SOLO con el JSON, sin explicaciones adicionales.

Texto del contrato:
{texto_limpio[:3000]}
"""
            respuesta = ollama.chat(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                options={
                    "temperature": 0.1,
                    "top_p": 0.9,
                    "repeat_penalty": 1.1,
                    "num_predict": 300
                }
            )
            content = respuesta['message']['content'].strip()
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if json_match:
                return json.loads(json_match.group(0))
            else:
                return {"error": "No se encontró JSON válido", "raw_response": content}
        except Exception as e:
            return {"error": f"Error procesando con Ollama: {str(e)}"}

# Procesamiento de PDFs
records = []
current_id = 1001
extractor = ContractExtractor()

for pdf_file in input_dir.glob("*.pdf"):
    banco = extract_banco_from_pdf(str(pdf_file))
    resultado_llm = extractor.extract_contract_data(str(pdf_file))

    nuevo_nombre = f"{banco} {fecha_proceso} {current_id}.pdf"
    destino_dir = output_base_dir / year / month / day / banco
    destino_dir.mkdir(parents=True, exist_ok=True)
    nuevo_path = destino_dir / nuevo_nombre

    try:
        shutil.move(str(pdf_file), str(nuevo_path))
    except Exception as e:
        print(f"Error moviendo archivo {pdf_file.name}: {e}")

    record = {
        "id": current_id,
        "archivo_original": pdf_file.name,
        "nuevo_nombre_archivo": nuevo_nombre,
        "banco": banco,
        "fecha_proceso": fecha_proceso
    }

    if 'error' in resultado_llm:
        record.update({
            "tasa_fwd": "ERROR",
            "valor_nominal_usd": "ERROR",
            "fecha_inicio": "ERROR",
            "error": resultado_llm.get("error", "Error desconocido")
        })
    else:
        record.update({
            "tasa_fwd": resultado_llm.get("tasa_fwd", ""),
            "valor_nominal_usd": resultado_llm.get("valor_nominal_usd", ""),
            "fecha_inicio": resultado_llm.get("fecha_inicio", ""),
            "error": ""
        })

    records.append(record)
    current_id += 1

# Guardar resultados en Excel
df = pd.DataFrame(records)
timestamp = datetime.now().strftime('%Y%m%d')
excel_path = excel_output_dir / f"forward_tracking_{timestamp}.xlsx"
df.to_excel(excel_path, index=False, engine='openpyxl')

print(f"Extracción completada. Resultados guardados en: {excel_path}")



Extracción completada. Resultados guardados en: Z:\17. Reporting Automation\Cartas NDFs\forward_tracking_20250730.xlsx


In [ ]:
import ollama
import json
import re
from pathlib import Path
import fitz  # PyMuPDF
import pdfplumber
from typing import Dict, Optional
import pandas as pd
from datetime import datetime
import pytesseract
from PIL import Image
import io

class ContractExtractor:
    def __init__(self, model="llama3.2:3b"):
        self.model = model
        # Configurar Tesseract (ajustar path según tu instalación)
        pytesseract.pytesseract.tesseract_cmd = r'C:\Users\ntorreslo\AppData\Local\Programs\Tesseract-OCR\tesseract'
    
    def extract_text_from_pdf(self, pdf_path: str, method="auto") -> str:
        """
        Extrae texto de PDF usando diferentes métodos incluyendo OCR
        """
        try:
            if method == "auto":
                # Primero intentar extracción normal
                text = self._extract_with_pdfplumber(pdf_path)
                
                # Si no hay texto o es muy poco, usar OCR
                if not text or len(text.strip()) < 100:
                    print("PDF parece escaneado, usando OCR...")
                    text = self._extract_with_ocr(pdf_path)
                
                return text
            
            elif method == "pdfplumber":
                return self._extract_with_pdfplumber(pdf_path)
            elif method == "pymupdf":
                return self._extract_with_pymupdf(pdf_path)
            elif method == "ocr":
                return self._extract_with_ocr(pdf_path)
            else:
                raise ValueError("Método no soportado")
        except Exception as e:
            print(f"Error extrayendo texto: {e}")
            return None
    
    def _extract_with_pdfplumber(self, pdf_path: str) -> str:
        """Extrae texto usando pdfplumber (mejor para tablas)"""
        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text
    
    def _extract_with_pymupdf(self, pdf_path: str) -> str:
        """Extrae texto usando PyMuPDF (más rápido)"""
        text = ""
        doc = fitz.open(pdf_path)
        for page_num in range(doc.page_count):
            page = doc[page_num]
            text += page.get_text() + "\n"
        doc.close()
        return text
    
    def _extract_with_ocr(self, pdf_path: str) -> str:
        """
        Extrae texto usando OCR (para PDFs escaneados)
        """
        text = ""
        try:
            doc = fitz.open(pdf_path)
            
            for page_num in range(doc.page_count):
                page = doc[page_num]
                
                # Convertir página a imagen
                pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # Aumentar resolución
                img_data = pix.tobytes("png")
                
                # Convertir a PIL Image
                image = Image.open(io.BytesIO(img_data))
                
                # Aplicar OCR
                page_text = pytesseract.image_to_string(image, lang='spa+eng')
                text += page_text + "\n"
                
                print(f"Página {page_num + 1} procesada con OCR")
            
            doc.close()
            return text
            
        except Exception as e:
            print(f"Error en OCR: {e}")
            print("Asegúrate de tener instalado Tesseract-OCR")
            return ""
    
    def extract_contract_data(self, pdf_path: str) -> Dict:
        """
        Extrae datos del contrato desde PDF
        """
        # Verificar si el archivo existe
        if not Path(pdf_path).exists():
            return {"error": f"Archivo no encontrado: {pdf_path}"}
        
        # Extraer texto del PDF (automáticamente detecta si necesita OCR)
        print("Extrayendo texto del PDF...")
        texto_contrato = self.extract_text_from_pdf(pdf_path, method="auto")
        
        if not texto_contrato:
            return {"error": "No se pudo extraer texto del PDF"}
        
        # Limpiar texto
        texto_limpio = self._clean_text(texto_contrato)
        
        # Crear prompt optimizado
        prompt = f"""
Eres un asistente especializado en análisis de contratos financieros.

Extrae ÚNICAMENTE estos campos del texto del contrato y responde con JSON limpio:

- banco: Nombre completo del banco (buscar en encabezados, firmas, "El Banco")
- tasa_fwd: Tasa Forward SIN PUNTOS DE MILES (formato: 4236,20 NO 4.236,20)
- valor_nominal_usd: Monto en USD (solo números, sin puntos ni comas)
- fecha_inicio: Fecha de operación/negociación (formato dd/mm/aaaa)

IMPORTANTE: 
- Para tasa_fwd: Si ves 4.236,20 debe ser 4236,20
- Para valor_nominal_usd: Si ves 2,000,000 debe ser 2000000
- Responde SOLO con el JSON, sin explicaciones adicionales.

Texto del contrato:
{texto_limpio[:3000]}
"""
        
        try:
            print("Procesando con Ollama...")
            respuesta = ollama.chat(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                options={
                    "temperature": 0.1,
                    "top_p": 0.9,
                    "repeat_penalty": 1.1,
                    "num_predict": 300
                }
            )
            
            # Limpiar y parsear respuesta
            content = respuesta['message']['content'].strip()
            resultado = self._parse_json_response(content)
            
            # Aplicar limpieza adicional
            if isinstance(resultado, dict) and 'error' not in resultado:
                resultado = self._clean_extracted_data(resultado)
            
            return resultado
            
        except Exception as e:
            return {"error": f"Error procesando con Ollama: {str(e)}"}
    
    def _clean_text(self, text: str) -> str:
        """Limpia el texto extraído del PDF"""
        # Eliminar saltos de línea excesivos
        text = re.sub(r'\n+', '\n', text)
        # Eliminar espacios excesivos
        text = re.sub(r' +', ' ', text)
        # Eliminar caracteres especiales problemáticos pero mantener los importantes
        text = re.sub(r'[^\w\s\.,:/\-()$áéíóúüñÁÉÍÓÚÜÑ]', '', text)
        return text.strip()
    
    def _clean_extracted_data(self, data: Dict) -> Dict:
        """
        Limpia y formatea los datos extraídos
        """
        # Limpiar tasa forward - ELIMINAR PUNTOS DE MILES
        if data.get('tasa_fwd'):
            tasa = str(data['tasa_fwd'])
            # Eliminar puntos de miles, mantener solo coma decimal
            tasa = re.sub(r'\.(?=\d{3})', '', tasa)  # Eliminar puntos de miles
            tasa = tasa.replace('.', ',')  # Convertir punto decimal a coma
            data['tasa_fwd'] = tasa
        
        # Limpiar valor nominal - eliminar puntos y comas
        if data.get('valor_nominal_usd'):
            valor = str(data['valor_nominal_usd'])
            valor = re.sub(r'[.,]', '', valor)  # Eliminar todos los puntos y comas
            data['valor_nominal_usd'] = valor
        
        return data
    
    def _parse_json_response(self, content: str) -> Dict:
        """Parsea la respuesta JSON del modelo"""
        try:
            # Buscar JSON en la respuesta
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                resultado = json.loads(json_str)
                
                # Validar campos requeridos
                campos_req = ['banco', 'tasa_fwd', 'valor_nominal_usd', 'fecha_inicio']
                if all(campo in resultado for campo in campos_req):
                    return resultado
                else:
                    return {"error": "Campos faltantes en JSON", "data": resultado}
            else:
                return {"error": "No se encontró JSON válido", "raw_response": content}
        except json.JSONDecodeError as e:
            return {"error": f"Error parseando JSON: {str(e)}", "raw_response": content}
    
    def save_to_excel(self, resultado: Dict, pdf_filename: str, output_dir: str = r"Z:\17. Reporting Automation\Cartas NDFs") -> str:
        """
        Guarda el resultado en Excel
        """
        try:
            # Crear directorio si no existe
            Path(output_dir).mkdir(parents=True, exist_ok=True)
            
            # Preparar datos para Excel
            if 'error' in resultado:
                # Si hay error, guardar información del error
                excel_data = {
                    'archivo': [pdf_filename],
                    'banco': ['ERROR'],
                    'tasa_fwd': ['ERROR'],
                    'valor_nominal_usd': ['ERROR'],
                    'fecha_inicio': ['ERROR'],
                    'error': [resultado.get('error', 'Error desconocido')],
                    'fecha_procesamiento': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')]
                }
            else:
                # Datos normales
                excel_data = {
                    'archivo': [pdf_filename],
                    'banco': [resultado.get('banco', '')],
                    'tasa_fwd': [resultado.get('tasa_fwd', '')],
                    'valor_nominal_usd': [resultado.get('valor_nominal_usd', '')],
                    'fecha_inicio': [resultado.get('fecha_inicio', '')],
                    'error': [''],
                    'fecha_procesamiento': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')]
                }
            
            # Crear DataFrame
            df = pd.DataFrame(excel_data)
            
            # Nombre del archivo Excel
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            excel_filename = f"contrato_extraido_{timestamp}.xlsx"
            excel_path = Path(output_dir) / excel_filename
            
            # Guardar en Excel
            df.to_excel(excel_path, index=False, engine='openpyxl')
            
            print(f"Resultado guardado en: {excel_path}")
            return str(excel_path)
            
        except Exception as e:
            error_msg = f"Error guardando en Excel: {str(e)}"
            print(error_msg)
            return error_msg
    
    def save_multiple_to_excel(self, resultados: Dict, output_dir: str = r"Z:\17. Reporting Automation\Cartas NDFs") -> str:
        """
        Guarda múltiples resultados en un solo archivo Excel
        """
        try:
            # Crear directorio si no existe
            Path(output_dir).mkdir(parents=True, exist_ok=True)
            
            # Preparar datos para Excel
            excel_data = {
                'archivo': [],
                'banco': [],
                'tasa_fwd': [],
                'valor_nominal_usd': [],
                'fecha_inicio': [],
                'error': [],
                'fecha_procesamiento': []
            }
            
            for filename, resultado in resultados.items():
                excel_data['archivo'].append(filename)
                excel_data['fecha_procesamiento'].append(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
                
                if 'error' in resultado:
                    excel_data['banco'].append('ERROR')
                    excel_data['tasa_fwd'].append('ERROR')
                    excel_data['valor_nominal_usd'].append('ERROR')
                    excel_data['fecha_inicio'].append('ERROR')
                    excel_data['error'].append(resultado.get('error', 'Error desconocido'))
                else:
                    excel_data['banco'].append(resultado.get('banco', ''))
                    excel_data['tasa_fwd'].append(resultado.get('tasa_fwd', ''))
                    excel_data['valor_nominal_usd'].append(resultado.get('valor_nominal_usd', ''))
                    excel_data['fecha_inicio'].append(resultado.get('fecha_inicio', ''))
                    excel_data['error'].append('')
            
            # Crear DataFrame
            df = pd.DataFrame(excel_data)
            
            # Nombre del archivo Excel
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            excel_filename = f"contratos_extraidos_{timestamp}.xlsx"
            excel_path = Path(output_dir) / excel_filename
            
            # Guardar en Excel
            df.to_excel(excel_path, index=False, engine='openpyxl')
            
            print(f"Resultados guardados en: {excel_path}")
            return str(excel_path)
            
        except Exception as e:
            error_msg = f"Error guardando múltiples resultados en Excel: {str(e)}"
            print(error_msg)
            return error_msg
    
    def process_multiple_pdfs(self, pdf_directory: str, save_excel: bool = True) -> Dict:
        """
        Procesa múltiples PDFs en un directorio y opcionalmente guarda en Excel
        """
        results = {}
        pdf_dir = Path(pdf_directory)
        
        if not pdf_dir.exists():
            return {"error": f"Directorio no encontrado: {pdf_directory}"}
        
        pdf_files = list(pdf_dir.glob("*.pdf"))
        
        for pdf_file in pdf_files:
            print(f"Procesando: {pdf_file.name}")
            results[pdf_file.name] = self.extract_contract_data(str(pdf_file))
        
        # Guardar en Excel si se solicita
        if save_excel and results:
            excel_path = self.save_multiple_to_excel(results)
            results['_excel_path'] = excel_path
        
        return results

# Funciones de conveniencia
def extraer_de_pdf(pdf_path: str, model="llama3.2:3b", save_excel: bool = True) -> Dict:
    """
    Función simple para extraer datos de un PDF y guardarlo en Excel
    """
    extractor = ContractExtractor(model=model)
    resultado = extractor.extract_contract_data(pdf_path)
    
    if save_excel:
        pdf_filename = Path(pdf_path).name
        excel_path = extractor.save_to_excel(resultado, pdf_filename)
        resultado['_excel_path'] = excel_path
    
    return resultado

def procesar_directorio(directorio: str, model="llama3.2:3b") -> Dict:
    """
    Procesa todos los PDFs de un directorio y guarda en Excel
    """
    extractor = ContractExtractor(model=model)
    return extractor.process_multiple_pdfs(directorio, save_excel=True)

# Ejemplo de uso
if __name__ == "__main__":
    # Procesar un PDF individual
    #pdf_path = r"Z:\11_Gersi\Contratos Firmados\CITI\Citi 26 Nov 19915 Firmado.pdf"
    pdf_path = r"Z:\17. Reporting Automation\Cartas NDFs\BANCO SANTANDER 17-06-25 1009.pdf"
    resultado = extraer_de_pdf(pdf_path, save_excel=True)
    
    print("Resultado:")
    print(json.dumps(resultado, indent=2, ensure_ascii=False))
    
    # Procesar directorio completo
    # resultados = procesar_directorio(r"Z:\11_Gersi\Contratos Firmados\CITI")
    # print("Todos los contratos procesados y guardados en Excel")

In [ ]:
import os
import re
import pandas as pd
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path 

# Configurar la ruta de Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\ntorreslo\AppData\Local\Programs\Tesseract-OCR\tesseract'

tasa_patterns = [
                   r"Forward Rate:\s*([\d.,]+)"
                 , r"Tasa Forward\s*[-:]?\s*([\d.,]+)"
                 , r"Tasa Forward\s*[-:]?\s*([\d.,]+)"
                 , r"(?i)(Tasa Forward|Forward Rate|Tasa):?\s*\$?\s*([\d\.,]+)"
                 , r'(?:Tasa Forward|Forward Rate|Tasa FW).*?[:\s].*?[\$\s]*([\d,]+\.?\d*)'
                 , r'(\d{4}[\.,]\d{2})\s*COP/USD'
                 , r'[\|\s](\d{4}[\.,]\d{2})[\|\s]'
                 , r'(\b\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})\b)'
                 ]

valor_patterns = [
    r"(?:Valor(?: Nocional| Nominal)?|Notional Amount|Monto en USD|Valor Negociado).*?:?\s*USD\s*([\d.,]+)",
    r"USD\s*([\d.,]+)",
    r"(?:nocional|nominal|monto|valor)\s*[:\-]?\s*\$?\s?\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?"
]

fecha_patterns = [
    r"(?:Fecha(?: de Celebraci[oó]n| de Vencimiento| de celebración)?|Trade Date).*?[:\-]?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{1,2}\s+de\s+\w{3,9}\.?(\s+de)?\s+\d{4})",
    r"\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b",
    r"\b\d{1,2}\s+de\s+\w{3,9}\.?(\s+de)?\s+\d{4}\b"
]

banco_patterns = [
                    r"JPMorgan, N\.A\."
                  , r"SCOTIABANK\.A\."
                  , r'(SCOTIABANK COLPATRIA S\.A\.)'
                  , r'(DAVIVIENDA)'
                  , r'(ITAÚ COLOMBIA S\.A\.)'
                  , r'(BANCOLOMBIA S\.A\.)'
                  , r'(JPMorgan)'
                  , r'(BANCO DE OCCIDENTE)'
                  , r'(BANCO SANTANDER[^,]*)'
                  , r'(CITIBANK COLOMBIA)'
                  
                  ]

def extract_data_from_pdf(file_path):
    images = convert_from_path(file_path
                               , dpi = 500
                               , poppler_path=r'C:\Poppler\poppler-24.08.0\Library\bin')  
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image, lang='spa')  # Cambia 'spa' por el idioma deseado

    # Define regex patterns for both Spanish and English documents
    patterns = {
        "fecha_celebracion": fecha_patterns[0],
        "tasa_forward": tasa_patterns[0],
        "valor_nominal": valor_patterns[0],
        "contraparte": banco_patterns[0]
        
    }
      

    data = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            data[key] = match.group(2).strip() if match.lastindex >= 2 else match.group(1).strip()
        else:
            data[key] = None

    # Clean the forward rate to retain only digits, commas, and periods
    if data["tasa_forward"]:
        data["tasa_forward"] = re.sub(r"[^\d,\.]", "", data["tasa_forward"])

    return data

def process_pdfs_in_directory(directory):
    records = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            data = extract_data_from_pdf(file_path)
            data["nombre_archivo"] = filename
            records.append(data)
    return records

# Directory containing the PDFs
directory = r"Z:/17. Reporting Automation/Cartas NDFs/Cartas sin firmas"
output_file = r"Z:/17. Reporting Automation/Cartas NDFs/forward_tracking_2.xlsx"

# Process the PDFs and get the data
records = process_pdfs_in_directory(directory)

# Create a DataFrame and save to Excel
df = pd.DataFrame(records)
df.to_excel(output_file, index=False)

print("The data has been successfully extracted and saved to forward_tracking.xlsx.")



4. Cambiar el nombres por el que ya se tiene con los bancos

5. Enviar al gerente respectivo a firmar

In [ ]:
import os
import win32com.client as win32
from datetime import datetime

# Ruta de los archivos PDF
pdf_folder = r"Z:\17. Reporting Automation\Cartas NDFs\Cartas para firmar"

# Obtener la fecha actual en formato dd-mm-aa
fecha_hoy = datetime.now().strftime("%d-%m-%y")

# Crear instancia de Outlook
olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

# Crear el correo
mailItem = olApp.CreateItem(0)
mailItem.Subject = f'Cartas NDFs {fecha_hoy}'
mailItem.BodyFormat = 1
mailItem.Body = """Cordial saludo, 

Adjunto confirmaciones para su firma. Mil gracias.

Saludos,
NTL

"""
mailItem.To = 'johann.najar@telefonica.com'
#mailItem.To = 'n.torres01@telefonica.com'

# Adjuntar todos los archivos PDF de la carpeta
for file in os.listdir(pdf_folder):
    if file.lower().endswith(".pdf"):
        full_path = os.path.join(pdf_folder, file)
        mailItem.Attachments.Add(full_path)

# Mostrar, guardar y enviar
mailItem.Display()
mailItem.Save()
mailItem.Send()


In [ ]:
import win32com.client
import os

try:
    # Inicializar la aplicación de Outlook
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.Folders("n.torres01@telefonica.com").Folders("Inbox").Folders("Spot")

    # Obtener todos los mensajes
    messages = inbox.Items
    messages.Sort("[ReceivedTime]", True)  # Ordenar por fecha descendente

    # Buscar el primer mensaje del remitente deseado
    latest_message = None
    for message in messages:
        if message.Class == 43:  # Asegura que sea un MailItem
            sender = message.SenderEmailAddress.lower()
            if "johann.najar@telefonica.com" in sender:
                latest_message = message
                break

    if latest_message:
        # Obtener el contenido del mensaje
        message_content = latest_message.Body

        # Ruta del archivo de salida
        file_path = r"Z:\17. Reporting Automation\Cartas NDFs\latest_email.txt"

        # Escribir el contenido en el archivo
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(message_content)

        print(f"✅ El contenido del último correo de Johann Najar fue guardado en:\n{file_path}")
    else:
        print("⚠️ No se encontró ningún correo reciente de johann.najar@telefonica.com en la bandeja de entrada.")

except Exception as e:
    print(f"❌ Ocurrió un error: {e}")



In [ ]:
import win32com.client
import os
from datetime import datetime, timedelta

try:
    # Inicializar la aplicación de Outlook
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.Folders("n.torres01@telefonica.com").Folders("Inbox").Folders("Spot")

    # Obtener la fecha de hace una semana
    last_week = datetime.now() - timedelta(days=7)

    # Obtener todos los mensajes
    messages = inbox.Items
    messages.Sort("[ReceivedTime]", True)  # Ordenar por fecha descendente

    # Buscar el primer mensaje con el asunto deseado de la última semana
    latest_message = None
    for message in messages:
        if message.Class == 43:  # Asegura que sea un MailItem
            if 'COMPRA DE DOLARES' in message.Subject.upper() and message.ReceivedTime >= last_week:
                latest_message = message
                break

    if latest_message:
        # Obtener el contenido del mensaje
        message_content = latest_message.Body

        # Ruta del archivo de salida
        file_path = r"Z:\17. Reporting Automation\Cartas NDFs\latest_email.txt"

        # Escribir el contenido en el archivo
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(message_content)

        print(f"✅ El contenido del último correo con asunto 'COMPRA DE DOLARES' fue guardado en:\n{file_path}")
    else:
        print("⚠️ No se encontró ningún correo reciente con el asunto 'COMPRA DE DOLARES' en la última semana.")

except Exception as e:
    print(f"❌ Ocurrió un error: {e}")
